In [ ]:
import os
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# CONFIGURACIÓN
escenarios = ["SSP126", "SSP245", "SSP585"]
ruta_base_general = "D:/TFG/Variables tratadas"
modelos = [
    "IPSL-CM6A-LR","HadGEM3-GC31-LL","CESM2-WACCM",
    "GFDL-CM4","MPI-ESM1-2-LR","NorESM2-LM",
    "ACCESS-CM2","EC-Earth3-Veg-LR","MIROC6","TaiESM1"
]
umbral_jet = 30.0  # m/s

# FUNCIÓN: asignar año invernal (DJF → invierno siguiente)
def asignar_invierno(ds):
    meses = ds['time'].dt.month
    años = ds['time'].dt.year
    winter_year = xr.where(meses == 12, años + 1, años)
    return ds.assign_coords(winter_year=("time", winter_year.data))

# FUNCIÓN: obtener media multimodelo DJF u250
def cargar_media_multimodelo(escenario):
    ruta_base = os.path.join(ruta_base_general, escenario)
    ua_all = []
    for modelo in modelos:
        ruta_modelo = os.path.join(ruta_base, modelo, f"{modelo}_ua_DJF.nc")
        if not os.path.exists(ruta_modelo):
            print(f"{modelo} no encontrado para {escenario}")
            continue

        ds = xr.open_dataset(ruta_modelo)
        varname = "ua" if "ua" in ds else list(ds.data_vars)[0]
        ua = ds[varname]

        if (ua.lon < 0).any():
            ua = ua.assign_coords(lon=((ua.lon + 360) % 360))
        ua = ua.sel(lat=slice(25, 70), lon=slice(280, 360))

        ua = asignar_invierno(ua)
        ua_djf_yearly = ua.groupby("winter_year").mean("time")

        ua_all.append(ua_djf_yearly.expand_dims(modelo=[modelo]))

    ua_multimodelo = xr.concat(ua_all, dim="modelo")
    ua_mean_multimodelo = ua_multimodelo.mean("modelo", skipna=True)

    u250 = ua_mean_multimodelo.sel(plev=25000, method="nearest")
    u250_djf_mean = u250.mean("winter_year")
    return u250_djf_mean

# CALCULAR MEDIAS
u250_126 = cargar_media_multimodelo("SSP126")
u250_245 = cargar_media_multimodelo("SSP245")
u250_585 = cargar_media_multimodelo("SSP585")

# Máscaras del jet
jet_126 = xr.where(u250_126 > umbral_jet, 1, np.nan)
jet_245 = xr.where(u250_245 > umbral_jet, 1, np.nan)
jet_585 = xr.where(u250_585 > umbral_jet, 1, np.nan)

# Enmascarar valores < 30 m/s
u250_126_masked = u250_126.where(u250_126 >= umbral_jet)
u250_245_masked = u250_245.where(u250_245 >= umbral_jet)
u250_585_masked = u250_585.where(u250_585 >= umbral_jet)

# FIGURA 1: TRES ESCENARIOS (1×3 PANELES)
fig1, axes1 = plt.subplots(1, 3, figsize=(14, 4),
                           subplot_kw={"projection": ccrs.PlateCarree()})
plt.subplots_adjust(wspace=0.08, bottom=0.15, top=0.85)

def plot_panel(ax, data, title, cmap, vmin=None, vmax=None):
    im = data.plot(
        ax=ax, transform=ccrs.PlateCarree(),
        cmap=cmap, vmin=vmin, vmax=vmax, add_colorbar=False
    )
    ax.coastlines(linewidth=0.7)
    ax.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax.set_extent([280, 354, 35, 70], crs=ccrs.PlateCarree())
    ax.set_title(title, fontsize=10)
    return im

# Escala común
vmin = min(float(u250_126_masked.min()), float(u250_245_masked.min()), float(u250_585_masked.min()))
vmax = max(float(u250_126_masked.max()), float(u250_245_masked.max()), float(u250_585_masked.max()))
niveles_jet = [30, 40]
anchos = [1.0, 1.5]

# Panel SSP126
im0 = plot_panel(axes1[0], u250_126_masked, "SSP1-2.6 (u ≥ 30 m/s)", "RdYlBu_r", vmin=vmin, vmax=vmax)
cs0 = axes1[0].contour(u250_126.lon, u250_126.lat, u250_126,
                       levels=niveles_jet, colors="black", linewidths=anchos,
                       transform=ccrs.PlateCarree())
axes1[0].clabel(cs0, fmt={30: "30 m/s", 40: "40 m/s"}, fontsize=7, colors="black")

# Panel SSP245
im1 = plot_panel(axes1[1], u250_245_masked, "SSP2-4.5 (u ≥ 30 m/s)", "RdYlBu_r", vmin=vmin, vmax=vmax)
cs1 = axes1[1].contour(u250_245.lon, u250_245.lat, u250_245,
                       levels=niveles_jet, colors="black", linewidths=anchos,
                       transform=ccrs.PlateCarree())
axes1[1].clabel(cs1, fmt={30: "30 m/s", 40: "40 m/s"}, fontsize=7, colors="black")

# Panel SSP585
im2 = plot_panel(axes1[2], u250_585_masked, "SSP5-8.5 (u ≥ 30 m/s)", "RdYlBu_r", vmin=vmin, vmax=vmax)
cs2 = axes1[2].contour(u250_585.lon, u250_585.lat, u250_585,
                       levels=niveles_jet, colors="black", linewidths=anchos,
                       transform=ccrs.PlateCarree())
axes1[2].clabel(cs2, fmt={30: "30 m/s", 40: "40 m/s"}, fontsize=7, colors="black")

# Colorbar común (centrada debajo de los tres paneles)
cbar_ax = fig1.add_axes([0.15, 0.1, 0.7, 0.04])
cbar = fig1.colorbar(im2, cax=cbar_ax, orientation="horizontal")
cbar.set_label("u (m/s)")

# Etiquetas lat/lon solo en primer panel (lat) y en todos los X (lon)
for i, ax in enumerate(axes1):
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                      linewidth=0.4, color='gray', alpha=0.5, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False
    gl.xlabel_style = {"size": 8}
    gl.ylabel_style = {"size": 8}
    gl.bottom_labels = True
    gl.left_labels = (i == 0)

plt.suptitle("Velocidad zonal media multimodelo (u250 ≥ 30 m/s)\npara los tres escenarios SSP durante DJF (2030–2100)",
             fontsize=12, y=0.94)

salida_fig1 = "C:/Users/jaime/Desktop/Universidad/TFG/Resultados/Comparacion_jet_mapas_SSP126_245_585.png"
plt.savefig(salida_fig1, dpi=300, bbox_inches="tight")
plt.show()
print(f"Figura 1 (mapas) guardada en:\n{salida_fig1}")


# FIGURA 2: COMPARACIÓN DE CONTORNOS DEL JET STREAM
fig2 = plt.figure(figsize=(6.5, 4))
ax = plt.axes(projection=ccrs.PlateCarree())

ax.coastlines(linewidth=0.7)
ax.add_feature(cfeature.BORDERS, linewidth=0.5)
ax.set_extent([280, 354, 35, 70], crs=ccrs.PlateCarree())
ax.gridlines(crs=ccrs.PlateCarree(), linewidth=0.4, color='gray',
             alpha=0.5, linestyle='--', draw_labels=True)

# Contornos superpuestos
cs_126 = ax.contour(u250_126.lon, u250_126.lat, u250_126,
                    levels=niveles_jet, colors="blue", linewidths=anchos,
                    transform=ccrs.PlateCarree())
cs_245 = ax.contour(u250_245.lon, u250_245.lat, u250_245,
                    levels=niveles_jet, colors="black", linewidths=anchos,
                    transform=ccrs.PlateCarree())
cs_585 = ax.contour(u250_585.lon, u250_585.lat, u250_585,
                    levels=niveles_jet, colors="red", linewidths=anchos,
                    transform=ccrs.PlateCarree())

ax.clabel(cs_126, fmt={30: "30", 40: "40"}, fontsize=7, colors="blue")
ax.clabel(cs_245, fmt={30: "30", 40: "40"}, fontsize=7, colors="black")
ax.clabel(cs_585, fmt={30: "30", 40: "40"}, fontsize=7, colors="red")

from matplotlib.lines import Line2D
legend_elements = [
    Line2D([0], [0], color="blue", lw=1.5, label="SSP1-2.6"),
    Line2D([0], [0], color="black", lw=1.5, label="SSP2-4.5"),
    Line2D([0], [0], color="red", lw=1.5, label="SSP5-8.5")
]
ax.legend(handles=legend_elements, loc="lower right", fontsize=8,
          frameon=True, framealpha=0.8)
plt.suptitle("Comparación de contornos del Jet Stream (30 y 40 m/s)", fontsize=11)

salida_fig2 = "C:/Users/jaime/Desktop/Universidad/TFG/Resultados/Comparacion_jet_contornos_SSP126_245_585.png"
plt.savefig(salida_fig2, dpi=300, bbox_inches="tight")
plt.show()

print(f"Figura 2 (contornos) guardada en:\n{salida_fig2}")


plt.suptitle("Posición y velocidad zonal media entre modelos del jet stream\npara 3 escenarios durante DJF (2030–2100)", fontsize=12, y=0.94)

salida_fig = "C:/Users/jaime/Desktop/Universidad/TFG/Resultados/Comparacion_jet_SSP126_245_585.png"
plt.savefig(salida_fig, dpi=300, bbox_inches="tight")
plt.show()

print(f"Figura comparativa guardada en:\n{salida_fig}")


In [ ]:
# EVOLUCIÓN TEMPORAL DEL JET STREAM (2030–2100)
def cargar_series_multimodelo(escenario):
    """
    Calcula la serie multimodelo DJF del viento zonal (u) a 250 hPa
    para un escenario dado (SSP245 o SSP585).

    Devuelve:
      - u250_djf_series: media multimodelo DJF por invierno
                         (dims: winter_year, lat, lon)
    """
    ruta_base = os.path.join(ruta_base_general, escenario)
    ua_all = []

    for modelo in modelos:
        ruta_modelo = os.path.join(ruta_base, modelo, f"{modelo}_ua_DJF.nc")
        if not os.path.exists(ruta_modelo):
            print(f"{modelo} no encontrado para {escenario}")
            continue

        ds = xr.open_dataset(ruta_modelo)
        varname = "ua" if "ua" in ds else list(ds.data_vars)[0]
        ua = ds[varname]

        # Asegura longitudes y dominio homogéneo
        if (ua.lon < 0).any():
            ua = ua.assign_coords(lon=((ua.lon + 360) % 360))
        ua = ua.sel(lat=slice(25, 70), lon=slice(280, 360))

        # Agrupa por invierno DJF (invierno siguiente)
        ua = asignar_invierno(ua)
        ua_djf_yearly = ua.groupby("winter_year").mean("time")

        ua_all.append(ua_djf_yearly.expand_dims(modelo=[modelo]))

    # Combinar modelos
    ua_multimodelo = xr.concat(ua_all, dim="modelo")

    # Media multimodelo
    ua_mean_multimodelo = ua_multimodelo.mean("modelo", skipna=True)

    # Seleccionar nivel de presión 250 hPa
    u250_djf_series = ua_mean_multimodelo.sel(plev=25000, method="nearest")

    return u250_djf_series

# CÁLCULO DE SERIES MULTIMODELO DJF
u250_126_series = cargar_series_multimodelo("SSP126")
u250_245_series = cargar_series_multimodelo("SSP245")
u250_585_series = cargar_series_multimodelo("SSP585")

print(" Series DJF multimodelo cargadas correctamente:")
print("   SSP1-2.6:", u250_126_series.dims)
print("   SSP2-4.5:", u250_245_series.dims)
print("   SSP5-8.5:", u250_585_series.dims)

# EVOLUCIÓN TEMPORAL Y TENDENCIA DEL JET STREAM (2030–2100)
import numpy as np
from scipy.stats import linregress

# Calcular la media espacial sobre el área del jet (u ≥ 30 m/s)
jet_series_126 = u250_126_series.where(u250_126_series >= 30).mean(dim=["lat", "lon"])
jet_series_245 = u250_245_series.where(u250_245_series >= 30).mean(dim=["lat", "lon"])
jet_series_585 = u250_585_series.where(u250_585_series >= 30).mean(dim=["lat", "lon"])

# Extraer arrays para regresión
years_126 = jet_series_126["winter_year"].values
years_245 = jet_series_245["winter_year"].values
years_585 = jet_series_585["winter_year"].values

vals_126 = jet_series_126.values
vals_245 = jet_series_245.values
vals_585 = jet_series_585.values

# Quitar posibles NaNs
mask_126 = ~np.isnan(vals_126)
mask_245 = ~np.isnan(vals_245)
mask_585 = ~np.isnan(vals_585)

# Calcular regresiones lineales
slope_126, intercept_126, r_126, p_126, _ = linregress(years_126[mask_126], vals_126[mask_126])
slope_245, intercept_245, r_245, p_245, _ = linregress(years_245[mask_245], vals_245[mask_245])
slope_585, intercept_585, r_585, p_585, _ = linregress(years_585[mask_585], vals_585[mask_585])

# Series ajustadas (líneas de tendencia)
trend_126 = intercept_126 + slope_126 * years_126
trend_245 = intercept_245 + slope_245 * years_245
trend_585 = intercept_585 + slope_585 * years_585

# Graficar la evolución temporal + tendencias + puntos
plt.figure(figsize=(8, 5))

# SSP1-2.6
plt.plot(years_126, vals_126, label="SSP1-2.6", color="#0072B2", linestyle="-", linewidth=1.2)
plt.plot(years_126, trend_126, color="#0072B2", linestyle=":", linewidth=2.0,
         label=f"Tendencia SSP1-2.6 ({slope_126*10:.2f} m/s por década)")

# SSP2-4.5
plt.plot(years_245, vals_245, label="SSP2-4.5", color="#009E73", linestyle="-", linewidth=1.2)
# plt.scatter(years_245, vals_245, color="black", s=20, alpha=0.7)  # puntos pequeños
plt.plot(years_245, trend_245, color="#009E73", linestyle=":", linewidth=2.0,
         label=f"Tendencia SSP2-4.5 ({slope_245*10:.2f} m/s por década)")

# SSP5-8.5
plt.plot(years_585, vals_585, label="SSP5-8.5", color="#E69F00", linestyle="-", linewidth=1.2)
# plt.scatter(years_585, vals_585, color="red", s=20, alpha=0.7)  # puntos pequeños
plt.plot(years_585, trend_585, color="#E69F00", linestyle=":", linewidth=2.0,
         label=f"Tendencia SSP5-8.5 ({slope_585*10:.2f} m/s por década)")
plt.xlabel("Año invernal (DJF)", fontsize=10)
plt.ylabel("u media sobre el jet (m/s)", fontsize=10)
plt.title("Evolución y tendencia de la velocidad zonal del Jet Stream DJF (2030–2100)\npara SSP1-2.6, SSP2-4.5 y SSP5-8.5", fontsize=12)
plt.legend(loc="upper left", fontsize=8)
plt.grid(True, linestyle="--", alpha=0.4)
plt.tight_layout()

salida_fig = "C:/Users/jaime/Desktop/Universidad/TFG/Resultados/evolución_tendencia_jet_escenarios.png"
plt.savefig(salida_fig, dpi=300, bbox_inches="tight")
plt.show()

# Mostrar estadísticas
print(f"SSP1-2.6 → pendiente = {slope_126:.4f} m/s/año  (p = {p_126:.3f})")
print(f"SSP2-4.5 → pendiente = {slope_245:.4f} m/s/año  (p = {p_245:.3f})")
print(f"SSP5-8.5 → pendiente = {slope_585:.4f} m/s/año  (p = {p_585:.3f})")


In [ ]:
# LATITUD MEDIA DEL JET STREAM (2030–2100)

def calcular_latitud_media_jet(u250_djf_series, umbral=30):
    """
    Calcula la latitud media del núcleo del jet para cada invierno DJF.
    - En cada longitud busca la latitud del máximo de u (donde u ≥ umbral).
    - Ignora longitudes sin valores válidos.
    - Devuelve una serie de latitud media por año.
    """
    latitudes_jet = []

    for year in u250_djf_series.winter_year.values:
        u_year = u250_djf_series.sel(winter_year=year)
        # Enmascara por umbral
        u_masked = u_year.where(u_year >= umbral)

        lat_max_list = []
        for lon in u_masked.lon.values:
            perfil = u_masked.sel(lon=lon)
            if np.all(np.isnan(perfil)):
                continue  # saltar columnas sin jet
            idx_max = int(perfil.argmax(dim="lat"))
            lat_max_list.append(perfil.lat.values[idx_max])

        # Si no hay longitudes válidas → NaN
        if len(lat_max_list) == 0:
            latitudes_jet.append(np.nan)
        else:
            latitudes_jet.append(np.nanmean(lat_max_list))

    return np.array(latitudes_jet)

# CÁLCULO DE SERIES PARA CADA ESCENARIO
lat_jet_126 = calcular_latitud_media_jet(u250_126_series, umbral=30)
lat_jet_245 = calcular_latitud_media_jet(u250_245_series, umbral=30)
lat_jet_585 = calcular_latitud_media_jet(u250_585_series, umbral=30)

# Años
years_126 = u250_126_series["winter_year"].values
years_245 = u250_245_series["winter_year"].values
years_585 = u250_585_series["winter_year"].values

# REGRESIÓN LINEAL DE LA LATITUD MEDIA
from scipy.stats import linregress

mask_126 = ~np.isnan(lat_jet_126)
mask_245 = ~np.isnan(lat_jet_245)
mask_585 = ~np.isnan(lat_jet_585)

slope_lat_126, intercept_lat_126, r_lat_126, p_lat_126, _ = linregress(years_126[mask_126], lat_jet_126[mask_126])
slope_lat_245, intercept_lat_245, r_lat_245, p_lat_245, _ = linregress(years_245[mask_245], lat_jet_245[mask_245])
slope_lat_585, intercept_lat_585, r_lat_585, p_lat_585, _ = linregress(years_585[mask_585], lat_jet_585[mask_585])

trend_lat_126 = intercept_lat_126 + slope_lat_126 * years_126
trend_lat_245 = intercept_lat_245 + slope_lat_245 * years_245
trend_lat_585 = intercept_lat_585 + slope_lat_585 * years_585

# COLORES TOL / OKABE–ITO
colors = {
    "SSP126": "#0072B2",  # azul petróleo
    "SSP245": "#009E73",  # verde oliva
    "SSP585": "#E69F00"   # naranja tostado
}

# GRÁFICO: EVOLUCIÓN LATITUD MEDIA DEL JET
plt.figure(figsize=(8, 5))

# SSP1-2.6
plt.plot(years_126, lat_jet_126, color=colors["SSP126"], linestyle="-", label="SSP1-2.6")
plt.plot(years_126, trend_lat_126, color=colors["SSP126"], linestyle=":", linewidth=2,
         label=f"Tendencia SSP1-2.6 ({slope_lat_126*10:.2f}° por década)")

# SSP2-4.5
plt.plot(years_245, lat_jet_245, color=colors["SSP245"], linestyle="-", label="SSP2-4.5")
plt.plot(years_245, trend_lat_245, color=colors["SSP245"], linestyle=":", linewidth=2,
         label=f"Tendencia SSP2-4.5 ({slope_lat_245*10:.2f}° por década)")

# SSP5-8.5
plt.plot(years_585, lat_jet_585, color=colors["SSP585"], linestyle="-", label="SSP5-8.5")
plt.plot(years_585, trend_lat_585, color=colors["SSP585"], linestyle=":", linewidth=2,
         label=f"Tendencia SSP5-8.5 ({slope_lat_585*10:.2f}° por década)")

plt.xlabel("Año invernal (DJF)", fontsize=10)
plt.ylabel("Latitud media del jet (°N)", fontsize=10)
plt.title("Evolución y tendencia de la latitud del Jet Stream en DJF (2030–2100)\npara SSP1-2.6, SSP2-4.5 y SSP5-8.5", fontsize=12)
plt.grid(True, linestyle="--", alpha=0.4)
plt.legend(loc="lower right", fontsize=8)
plt.tight_layout()

salida_fig = "C:/Users/jaime/Desktop/Universidad/TFG/Resultados/evolución_tendencia_latitud_jet_SSP126_245_585.png"
plt.savefig(salida_fig, dpi=300, bbox_inches="tight")
plt.show()

print(f" Figura guardada en: {salida_fig}")
print(f"SSP1-2.6 → pendiente = {slope_lat_126:.4f} °/año (p = {p_lat_126:.3f})")
print(f"SSP2-4.5 → pendiente = {slope_lat_245:.4f} °/año (p = {p_lat_245:.3f})")
print(f"SSP5-8.5 → pendiente = {slope_lat_585:.4f} °/año (p = {p_lat_585:.3f})")
